## Retirever & Chain with Langchain

In [4]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Automating_records.pdf")
docs = loader.load()
docs

Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 62 0 (offset 0)


[Document(metadata={'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creator': 'pdftk 1.41 - www.pdftk.com', 'creationdate': "D:20180729005033Z00'00'", 'moddate': "D:20180729005033Z00'00'", 'source': 'Automating_records.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Automating Records \nManagement\nChristine Shervington\nIntroduction\nThis paper will give an outline of an automated current records \nmanagement programme in an Australian institution, the University of \nWestern A ustralia. The University A rchivist is involved in the organisation \nand care of both the archival records, and the current records management \nprogramme and office, a position which provides an overview of the day to \nday administrative needs placed on the records as well as the historical \nexpectations held by administrators and researchers.\nRecent literature1 on automation of current and non-current records \nmanagement has made much of the fact that traditionally, archival find

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunk_docs = text_splitter.split_documents(docs)

chunk_docs

[Document(metadata={'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creator': 'pdftk 1.41 - www.pdftk.com', 'creationdate': "D:20180729005033Z00'00'", 'moddate': "D:20180729005033Z00'00'", 'source': 'Automating_records.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Automating Records \nManagement\nChristine Shervington\nIntroduction\nThis paper will give an outline of an automated current records \nmanagement programme in an Australian institution, the University of \nWestern A ustralia. The University A rchivist is involved in the organisation \nand care of both the archival records, and the current records management \nprogramme and office, a position which provides an overview of the day to \nday administrative needs placed on the records as well as the historical \nexpectations held by administrators and researchers.\nRecent literature1 on automation of current and non-current records \nmanagement has made much of the fact that traditionally, archival find

In [6]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="gemma:2b")  

db = FAISS.from_documents(chunk_docs, embeddings)
db


In [7]:
query = " Another refinement which is currently under consideration is the institution of a bar-code checking system."
result = db.similarity_search(query)
print(result[1].page_content)

138 AUTOMATING RECORDS MANAGEMENT
These systems may be more powerful in that they access material at 
document level but the running costs of such operations are prohibitive in 
an education institution facing financial cut-backs annually.
There are however, several enhancements or variations which could have 
been made if a package other than DATATR1EVE had been used. For 
instance, although it was originally planned to include a procedure to 
maintain statistical details such as title of officer and date of recent use of 
each file, this proved too expensive in computer time because of the need 
for the programme to interact with the master file.
Consequently, although this feature is recognised by experts in the field 
as one of the great benefits of automation8 it has been abandoned in this 
instance. However, it is still possible to identify recent users, as officers’ 
titles are listed on the face sheet as a directive when the file is checked out.


In [8]:
## Designing ChatPrompt Template 
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
                                          Answer the following question based on the context provided.
                                          The answer should be concise and relevant to the question.
                                          <contect> {context} </contect>
                                          Question: {input}
                                          """)

In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model="gemma:2b")
llm

C:\Users\USER\AppData\Local\Temp\ipykernel_13720\2354740381.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma:2b")


Ollama(model='gemma:2b')

In [10]:
## Chain 
from langchain.chains.combine_documents import create_stuff_documents_chain
doc_chain = create_stuff_documents_chain(llm, prompt=prompt)
doc_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n                                          Answer the following question based on the context provided.\n                                          The answer should be concise and relevant to the question.\n                                          <contect> {context} </contect>\n                                          Question: {input}\n                                          '), additional_kwargs={})])
| Ollama(model='gemma:2b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [11]:
retirever = db.as_retriever()
retirever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A4B214A3C0>, search_kwargs={})

In [12]:
from langchain.chains import create_retrieval_chain
reteirevel_chain = create_retrieval_chain(retirever,doc_chain)


In [17]:
reteirevel_chain.invoke({"input": "University Archivist"})


{'input': 'University Archivist',
 'context': [Document(id='fa66940c-2e13-4d40-9d83-638d5dac896c', metadata={'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creator': 'pdftk 1.41 - www.pdftk.com', 'creationdate': "D:20180729005033Z00'00'", 'moddate': "D:20180729005033Z00'00'", 'source': 'Automating_records.pdf', 'total_pages': 15, 'page': 11, 'page_label': '12'}, page_content='successful.\nAcknowledgements\nMr Noel Rogers, Records Manager, University of Sydney, for providing \ndocumentation and discussing the principles of keyword indexing as \napplied to University records.\nMrs Carole Herriman of Western Australian Regional Computing \nCentre for surveying needs and drawing up computer specifications and \nMrs Pene Duffy of Western Australian Regional Computer Centre for \nundertaking the programming.'),
  Document(id='0d6e9324-ae8f-439e-8595-8b51b94c5c76', metadata={'producer': 'Mac OS X 10.11.6 Quartz PDFContext', 'creator': 'pdftk 1.41 - www.pdftk.com', 'creationdate': "D:20180